In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

c:\Users\amate\Desktop\engineering\GOSMIC_GOZMIC\vits-finetuning\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=4)
           2	LOAD_FAST(arg=0, lineno=7)
           4	LOAD_ATTR(arg=0, lineno=7)
           6	LOAD_CONST(arg=1, lineno=7)
           8	BINARY_SUBSCR(arg=None, lineno=7)
          10	STORE_FAST(arg=4, lineno=7)
          12	LOAD_CONST(arg=2, lineno=8)
          14	STORE_FAST(arg=5, lineno=8)
          16	LOAD_GLOBAL(arg=1, lineno=9)
          18	LOAD_GLOBAL(arg=2, lineno=9)
          20	LOAD_FAST(arg=4, lineno=9)
          22	CALL_FUNCTION(arg=1, lineno=9)
          24	CALL_FUNCTION(arg=1, lineno=9)
          26	GET_ITER(arg=None, lineno=9)
>         28	FOR_ITER(arg=152, lineno=9)
          30	STORE_FAST(arg=6, lineno=9)
          32	LOAD_FAST(arg=0, lineno=10)
          34	LOAD_FAST(arg=6, lineno=10)
          36	BINARY_SUBSCR(arg=None, lineno=10)
          38	STORE_FAST(arg=7, lineno=10)
          40	LOAD_FAST(arg=1, lineno=11)
          42	LOAD_FAST(arg=6, lineno=11)
          44	BINARY_SUBSCR(arg=None, line

## Single Speaker

In [2]:
hps = utils.get_hparams_from_file("configs/XXX.json")

FileNotFoundError: [Errno 2] No such file or directory: 'configs/XXX.json'

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("/path/to/model.pth", net_g, None)

In [ ]:
stn_tst = get_text("こんにちは", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

## Multiple Speakers

In [2]:
hps = utils.get_hparams_from_file("config.json")

In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("pretrained_model\G_4000_42_Einstein.pth", net_g, None)

INFO:root:Loaded checkpoint 'pretrained_model\G_4000_42_Einstein.pth' (iteration 667)


In [10]:
stn_tst = get_text("これ全部CPUでやっているのにめちゃくちゃ速い。これすごくないですか？", hps)
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    sid = torch.LongTensor([42])
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=0.8)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [ ]:
stn_tst = get_text("こんにちは", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([4]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))